#import packages

In [ ]:
from selenium import webdriver
import bs4
import requests
import pandas as pd
import numpy as np
import time

#Collect website link

In [ ]:
#use selenium to open shopee's mom and kid category
driver = webdriver.Chrome(executable_path = r'C:\Users\paint\Desktop\shopee\chromedriver')
driver.get('https://shopee.co.th/')
time.sleep(5)

thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
time.sleep(5)

close_button = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_button.click()
time.sleep(5)

mom_kid_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div[1]/div/div/div[2]/div/div[1]/ul/li[9]/div/a[1]/div/div[2]/div')
mom_kid_button.click()
time.sleep(5)

seeall_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[1]/a[2]/div')
seeall_button.click()
time.sleep(5)

In [ ]:
#zoom out and scroll down to load all element
driver.execute_script("document.body.style.zoom='10%'")
time.sleep(5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10)

#collect all shop link
data = driver.page_source
soup = bs4.BeautifulSoup(data)

url_list = []
for data in driver.find_elements_by_xpath('//a[@class="full-brand-list-item__brand-cover-image"]'):
    url_list.append(data.get_attribute('href'))

#Scraping data

In [ ]:
shopee_data = pd.DataFrame(columns=['name', 'price', 'quantity', 'shop'])

In [ ]:
page = 0
scrape_url = []
while page < 10:
    url_extended = url_list[0]+'?page=' + str(page) + '&sortBy=pop'
    scrape_url.append(url_extended)
    page += 1

In [ ]:
count = 1
for url in scrape_url:
    products_list = []
    prices_list = []
    quantities_list = []

    driver = webdriver.Chrome()
    driver.get(url)

    #add cookie
    for i in cookies:
        driver.add_cookie(i)
    driver.refresh()
    time.sleep(10)

    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(5)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    a = soup.find_all('div', {'class', 'SiHtSX'})

    for b in a:
        if b.find('div', {'class': 'DI9B6s n9ce8z tHUXoY'}) is not None:
            products_list.append(b.find('div', {'class': 'DI9B6s n9ce8z tHUXoY'}).text)
        elif b.find('div', {'class': 'DI9B6s tHUXoY'}) is not None:
            products_list.append(b.find('div', {'class': 'DI9B6s tHUXoY'}).text)
        else:
            products_list.append(None)

        if b.find('div', {'class': '_5nsylV g5TpxV'}) is not None:
            prices_list.append(b.find('div', {'class': '_5nsylV g5TpxV'}).text)
        elif b.find('div', {'class': '_5nsylV o52c1W'}) is not None:
            prices_list.append(b.find('div', {'class': '_5nsylV o52c1W'}).text)
        elif b.find('div', {'class': '_5nsylV A15K0C'}) is not None:
            prices_list.append(b.find('div', {'class': '_5nsylV A15K0C'}).text)
        else:
            prices_list.append(None)

        if b.find('div', {'class': 'ktCe9A BlWcUe'}) is not None:
            quantities_list.append(b.find('div', {'class': 'ktCe9A BlWcUe'}).text)
        elif b.find('div', {'class': 'ktCe9A f3IWem'}) is not None:
            quantities_list.append(b.find('div', {'class': 'ktCe9A f3IWem'}).text)
        elif b.find('div', {'class': 'ktCe9A'}) is not None:
            quantities_list.append(b.find('div', {'class': 'ktCe9A'}).text)
        else:
            quantities_list.append(None)

    shop_data = pd.DataFrame([products_list, prices_list, quantities_list])
    shop_data = shop_data.transpose()
    shop_data.columns = ['name', 'price', 'quantity']
    shop_data['shop'] = soup.find('h1', {'class': 'section-seller-overview-horizontal__portrait-name'}).text

    shopee_data = pd.concat([shopee_data, shop_data])

    print('complete: ', count)
    count += 1
    #time.sleep(10)

print('complete all')

#Clean data

In [ ]:
#remove duplicate rows
shopee_data_nodupe = shopee_data.drop_duplicates()
shopee_data_clean = shopee_data_nodupe

In [ ]:
#clean data
shopee_data_clean['price'] = shopee_data_clean['price'].str.replace('฿', '')
shopee_data_clean['price'] = shopee_data_clean['price'].str.replace(',', '')

shopee_data_clean['quantity'] = shopee_data_clean['quantity'].str.replace('ขายแล้ว', '')
shopee_data_clean['quantity'] = shopee_data_clean['quantity'].str.replace('ชิ้น', '')
shopee_data_clean['quantity'] = shopee_data_clean['quantity'].str.replace('.', '')
shopee_data_clean['quantity'] = shopee_data_clean['quantity'].str.replace('พัน', '000')

In [ ]:
#split price data
shopee_data_clean[['min_price', 'max_price']] = shopee_data_clean['price'].str.split(pat='-', expand=True)

#change column variable type to numeric
shopee_data_clean['quantity'] = pd.to_numeric(shopee_data_clean['quantity'])
shopee_data_clean['min_price'] = pd.to_numeric(shopee_data_clean['min_price'])
shopee_data_clean['max_price'] = pd.to_numeric(shopee_data_clean['max_price'])

#create avg_price column
shopee_data_clean['avg_price'] = np.where(shopee_data_clean['max_price'].notnull(), (shopee_data_clean['min_price']+shopee_data_clean['max_price'])/2, shopee_data_clean['min_price'])

In [ ]:
#reset index
shopee_data_clean = shopee_data_clean.reset_index(drop = True)